In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

### Reading all the focal brands

In [ ]:
with open('top_brands.pickle', 'rb') as file:
    focal_brands = pickle.load(file)

focal_brands

### Reading the Data for Local Reviews of all brands having social data

In [ ]:
with open('brand_visit_local_reviews.pickle', 'rb') as file:
    brand_visit_local_reviews = pickle.load(file)

brand_visit_local_reviews.head()

### Read the distance results

In [ ]:
distance_results = pd.read_csv('data/distance_results.csv')
distance_results.head()

### Extracting the focal brands data to extract first neighbor and second neighbors

Getting the list of all focal brands

In [36]:
focal_brands_list = focal_brands['BRANDS'].to_list()

Getting visit and local review dataset for each focal brand

In [37]:
focal_brands_visit_local_reviews = brand_visit_local_reviews[brand_visit_local_reviews['Name'].isin(focal_brands_list)]

In [38]:
def extract_first_second_degree_neighbors(foc_df):
    first_neighbors = []
    second_neighbors = {}
    
    # Extracting all first degree neighbors
    focal_brand_name = foc_df['Name'].iloc[0]
    first_neighbors = distance_results[(distance_results['From_PLACEKEY'] == foc_df['PLACEKEY'].iloc[0]) & (distance_results['Distance_km']<=16.0934)]['To_PLACEKEY'].to_list()
    
    # Extracting all second degree Neighbors and saving them as dictionary for relevant first degree neighbors
    for j in tqdm(range(len(first_neighbors))):
        second_neighbors_temp = distance_results[(distance_results['From_PLACEKEY'] == first_neighbors[j]) & (distance_results['Distance_km']<=16.0934)]['To_PLACEKEY'].to_list()
        second_neighbors_temp = [second_neib for second_neib in second_neighbors_temp if second_neib not in first_neighbors]
        second_neighbors[first_neighbors[j]] = list(set(second_neighbors_temp))
        
    return pd.Series([focal_brand_name, first_neighbors, second_neighbors])

In [ ]:
focal_brands_first_second_degree_neighbors = None

for i in range(len(focal_brands_list)):
    print('Executing for Brand:', focal_brands_list[i])
    
    foc_df = focal_brands_visit_local_reviews[focal_brands_visit_local_reviews['Name'] == focal_brands_list[i]]
    first_second_neib_df = foc_df.groupby(['PLACEKEY'])[['PLACEKEY', 'Name']].apply(extract_first_second_degree_neighbors)
    
    if i == 0: # For the very first time, store the dataframe. For any next iterations, just perform the concatenation
        focal_brands_first_second_degree_neighbors = first_second_neib_df
    else:
        focal_brands_first_second_degree_neighbors = pd.concat([focal_brands_first_second_degree_neighbors, first_second_neib_df], axis=0)

In [40]:
with open('focal_brands_first_second_degree_neighbors.pickle', 'wb') as file:
    pickle.dump(focal_brands_first_second_degree_neighbors, file)